# Toy example to demonstrate the use of mescal

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from mescal import *
import bw2data as bd
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
ei_version = '3.10.1'

In [ ]:
# AMPL licence
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL license file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [ ]:
path_model = './data/esm/' # Path to the energy system model
path_model_lca = './data/esm/lca/'
path_inputs = './data/lca/' # Path to the LCA data
path_results = './results/' # Path to the results

## Generating LCA impact scores

In [ ]:
mapping = pd.read_csv(path_inputs+'mapping.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')
model = pd.read_csv(path_inputs+'model.csv')

In [ ]:
df_si_paper = pd.merge(mapping, unit_conversion, on=['Name', 'Type']).drop(columns=['Unit', 'Database']).rename(columns={'LCA': 'LCA unit', 'ESM': 'ESM unit', 'Name': 'Technology', 'Product': 'LCI dataset product', 'Activity': 'LCI dataset activity', 'Location': 'LCI dataset location', 'Value': 'Conversion factor', 'Type': 'Life cycle phase'})
df_si_paper['Technology'] = df_si_paper['Technology'].apply(lambda x: tech_name_dict[x] if x in tech_name_dict.keys() else x)
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_CONNECTION', 'Onshore wind (connection part)')
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_TURBINE', 'Onshore wind (turbine part)')
# df_si_paper.to_csv('./data/lca/si_paper.csv', index=False)

In [ ]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version} backup')

In [ ]:
name_main_database_2050 = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_2050'
esm_db_name = 'Tatooine'

In [ ]:
main_db_2050 = Database(name_main_database_2050, create_pickle=True)

In [ ]:
# Add CPC categories to the main database
main_db_2050.add_CPC_categories()

In [ ]:
ranking_best_ecoinvent_locations = ['GLO', 'RoW']

In [ ]:
esm = PathwayESM(
    # Mandatory inputs
    time_steps=[
        {'year': 2050, 'model': model, 'lifetime':lifetime, 'main_database': main_db_2050},
    ],
    mapping=mapping.copy(deep=True),
    unit_conversion=unit_conversion,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    esm_db_name=esm_db_name,
    esm_location='GLO',

    # Optional inputs
    technology_compositions=techno_compositions,
    efficiency=efficiency,
    regionalize_foregrounds=False,
    locations_ranking=ranking_best_ecoinvent_locations,
    results_path_file=path_results,
    biosphere_db_name='ecoinvent-3.10.1-biosphere',
)

In [ ]:
esm.check_inputs()

In [ ]:
missing_flows = main_db_2050.test_mapping_file(esm.mapping[esm.mapping.Year == 2050])

In [ ]:
esm.create_esm_database()

In [ ]:
methods = ['IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'IMPACT World+ Damage 2.1 for ecoinvent v3.10']

In [ ]:
# Contribution analysis of elementary flows for total human health and total ecosystem quality
R_long, contrib_analysis = esm.compute_impact_scores(
    methods=methods,
    impact_abbrev=impact_abbrev,
    contribution_analysis='emissions',
    contribution_analysis_limit_type='number',
    contribution_analysis_limit=10,
)
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]
contrib_analysis.to_csv(f'{path_results}contribution_analysis.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [ ]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': '2050',
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [ ]:
specific_lcia_abbrev = ['m_CCS', 'TTHH', 'TTEQ']

In [ ]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='techs_lca',
)

In [ ]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='objectives_lca',
)

## First iteration: single objective optimization with a snapshot ESM

In [ ]:
impact_scores = pd.read_csv(path_results+'impact_scores.csv')
impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

In [ ]:
list_esm_results_f_mult = []
list_esm_results_annual_res = []
list_esm_results_annual_prod = []
list_main_variables_results = []

for obj in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:

    es = run_esm(obj+'[2050]', returns='model')
    results = es.calc()

    df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
        df_results=results,
        df_impact_scores=impact_scores,
        impact_category=list(impact_scores['Impact_category'].unique()),
    )

    df_f_mult['Run'] = obj
    df_annual_prod['Run'] = obj
    df_annual_res['Run'] = obj

    # we only keep the results for 2050
    df_f_mult = df_f_mult[df_f_mult['index1'] == 2050]
    df_annual_prod = df_annual_prod[df_annual_prod['index1'] == 2050]
    df_annual_res = df_annual_res[df_annual_res['index1'] == 2050]

    df_f_mult.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
    df_annual_prod.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
    df_annual_res.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)

    # we keep the first value (i.e., the one for 2050)
    total_cost = results.variables['TotalCost'].TotalCost.values[0]
    total_m_ccs = results.variables['TotalLCIA_m_CCS'].TotalLCIA_m_CCS.values[0]
    total_tthh = results.variables['TotalLCIA_TTHH'].TotalLCIA_TTHH.values[0]
    total_tteq = results.variables['TotalLCIA_TTEQ'].TotalLCIA_TTEQ.values[0]

    list_main_variables_results.append([obj, total_cost, total_m_ccs, total_tthh, total_tteq])

    list_esm_results_f_mult.append(df_f_mult)
    list_esm_results_annual_prod.append(df_annual_prod)
    list_esm_results_annual_res.append(df_annual_res)

esm_results_f_mult = pd.concat(list_esm_results_f_mult)
esm_results_annual_prod = pd.concat(list_esm_results_annual_prod)
esm_results_annual_res = pd.concat(list_esm_results_annual_res)
main_variables_results = pd.DataFrame(data=list_main_variables_results, columns=['Objective', 'TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ'])

In [ ]:
# lyrio = results.parameters['layers_in_out'].reset_index()
# lyrio = lyrio[lyrio.layers_in_out != 0].drop(columns=['Run']).rename({'index0': 'Name', 'index1': 'Flow', 'layers_in_out': 'Amount'}, axis=1)
# lyrio.to_csv(path_inputs+'model.csv', index=False)

## Create new LCI datasets from the ESM results

In [ ]:
esm_results = esm_results_annual_prod.rename(columns={'Annual_Prod': 'Production'})[['Name', 'Production', 'Run']]
esm_results = esm_results.merge(esm_results_f_mult[['Name', 'F_Mult', 'Run']], on=['Name', 'Run'], how='left').rename(columns={'F_Mult': 'Capacity'})

### With harmonization

In [ ]:
esm.esm_location = 'WAF' # we want to take the WAF electricity markets as a dataset template
for run in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:
    esm.create_new_database_with_esm_results(
        esm_results=esm_results[esm_results['Run'] == run],
        remove_background_construction_flows=False,
        harmonize_with_esm=True,
        esm_results_db_name=f'{esm.esm_db_name}_2050_results_{run.replace("Total", "").replace("LCIA_", "").lower()}',
    )

### Without harmonization

In [ ]:
for run in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:
    esm.create_new_database_with_esm_results(
        esm_results=esm_results[esm_results['Run'] == run],
        remove_background_construction_flows=False,
        harmonize_with_esm=False,
        esm_results_db_name=f'{esm.esm_db_name}_2050_results_{run.replace("Total", "").replace("LCIA_", "").lower()}_wo_harmonization',
    )

### Generate new metrics from the SOO results

In [ ]:
locations = [
    'BRA',
    'CAN',
    'CEU',
    'CHN',
    'EAF',
    'INDIA',
    'INDO',
    'JAP',
    'KOR',
    'ME',
    'MEX',
    'NAF',
    'OCE',
    'RCAM',
    'RSAF',
    'RSAM',
    'RSAS',
    'RUS',
    'SAF',
    'SEAS',
    'STAN',
    'TUR',
    'UKR',
    'USA',
    'WAF',
    'WEU',
    'World',
    'GLO',
    'RoW',
]  # list of premise locations --> all these electricity markets will be updated with the ESM results

In [ ]:
methods = ['IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'IMPACT World+ Damage 2.1 for ecoinvent v3.10']

In [ ]:
for run in ['TotalLCIA_m_CCS', 'TotalCost', 'TotalLCIA_TTEQ', 'TotalLCIA_TTHH']:

    for harmonized in [True, False]:

        if harmonized:
            suffix = f'{run.replace("Total", "").replace("LCIA_", "").lower()}'
        else:
            suffix = f'{run.replace("Total", "").replace("LCIA_", "").lower()}_wo_harmonization'

        esm_results_db_name = f'{esm_db_name}_2050_results_{suffix}'
        new_esm_db_name = f'{esm_db_name}_2050_{suffix}'
        impact_scores_df_name = f'impact_scores_{suffix}'
        dat_file_name = f'techs_lca_{suffix}'

        main_db_2050 = Database(name_main_database_2050, create_pickle=True)
        main_db_2050.add_CPC_categories()

        esm = PathwayESM(
            # Mandatory inputs
            time_steps=2*[{'year': 2050, 'model': model, 'main_database': main_db_2050}],
            mapping=mapping.copy(deep=True),
            unit_conversion=unit_conversion,
            mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
            esm_db_name=esm_db_name,
            esm_location='WAF',

            # Optional inputs
            technology_compositions=techno_compositions,
            lifetime=lifetime,
            efficiency=efficiency,
            regionalize_foregrounds=False,
            locations_ranking=ranking_best_ecoinvent_locations,
            results_path_file=path_results,
            biosphere_db_name='ecoinvent-3.10.1-biosphere',
        )

        # Connecting the results of the ESM to the ESM database
        esm.connect_esm_results_to_database(
            esm_results_db_name=esm_results_db_name,
            specific_db_name=f'{esm.esm_db_name}_2050',
            create_new_db=True,
            new_db_name=new_esm_db_name,
            locations=locations,
            update_exchanges_based_on_activity_name=False,
        )

        # Connecting the results of the ESM to the main database (i.e., premise database)
        esm.connect_esm_results_to_database(
            esm_results_db_name=esm_results_db_name,
            create_new_db=True,
            new_db_name=f'{esm.main_database_name}_connected_{suffix}',
            locations=locations,
            update_exchanges_based_on_activity_name=False,
        )

        # relinking the newly created ESM database to the updated main database
        Database(new_esm_db_name).relink(
            name_database_unlink=esm.main_database_name,
            name_database_relink=f'{esm.main_database_name}_connected_{suffix}',
            write=True,
        )

        esm.time_steps = [{'year': 2050, 'model': model, 'main_database': main_db_2050}]  # only compute lca metrics once

        # Computing the LCA indicators with the new ESM database
        R_long, contribution_analysis = esm.compute_impact_scores(
            esm_db_name=new_esm_db_name,
            methods=methods,
            impact_abbrev=impact_abbrev,
            specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
        )

        R_long.to_csv(f'{path_results}2050/{impact_scores_df_name}.csv', index=False)

        # R_long = pd.read_csv(f'{path_results}2050/{impact_scores_df_name}.csv')

        # Create .dat file
        esm.normalize_lca_metrics(
            R=R_long,
            mip_gap=1e-6,
            lcia_methods=methods,
            specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
            impact_abbrev=impact_abbrev,
            path=path_model_lca+'esm_results/',
            file_name=dat_file_name,
        )

### Second iteration: re-run the ESM with the first iteration results fed back in the LCI database

In [ ]:
for obj in ['TotalLCIA_m_CCS', 'TotalCost', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:

    for background in ['base', 'esm_not_harmonized', 'esm_harmonized']:

        if background == 'base':
            impact_scores = pd.read_csv(f'./results/impact_scores.csv')
        elif background == 'esm_not_harmonized':
            impact_scores = pd.read_csv(f'./results/2050/impact_scores_{obj.replace("Total", "").replace("LCIA_", "").lower()}_wo_harmonization.csv')
        elif background == 'esm_harmonized':
            impact_scores = pd.read_csv(f'./results/2050/impact_scores_{obj.replace("Total", "").replace("LCIA_", "").lower()}.csv')
        else:
            raise ValueError("Background must be one of 'base', 'esm_not_harmonized', or 'esm_harmonized'.")

        impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

        es = run_esm(obj+'[2050]', scenario=True, lca_metrics_background=background, returns='model')
        results = es.calc()

        df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
            df_results=results,
            df_impact_scores=impact_scores,
            impact_category=list(impact_scores['Impact_category'].unique()),
            specific_year=2050,
        )

        df_f_mult['Run'] = f'{obj}_{background}'
        df_annual_prod['Run'] = f'{obj}_{background}'
        df_annual_res['Run'] = f'{obj}_{background}'

        df_f_mult.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
        df_annual_prod.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
        df_annual_res.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)

        list_esm_results_f_mult.append(df_f_mult)
        list_esm_results_annual_prod.append(df_annual_prod)
        list_esm_results_annual_res.append(df_annual_res)

esm_results_f_mult = pd.concat(list_esm_results_f_mult)
esm_results_annual_prod = pd.concat(list_esm_results_annual_prod)
esm_results_annual_res = pd.concat(list_esm_results_annual_res)

To compare those results to what would typically be obtained with an ESM not coupled to LCA, we compute the impact obtained from direct CO2 emissions only.

In [ ]:
esm_db = Database('Tatooine_2050')

In [ ]:
df_flows_set_to_zero = pd.read_csv(path_results+'2050/removed_flows_list.csv')

In [ ]:
# Retrieving direct CO2 emissions
direct_co2_emissions_dict = {}

for tech in mapping.Name.unique():
    direct_co2_emissions_dict[tech] = 0

for tech in df_flows_set_to_zero.Name.unique():
    act_name_list =  df_flows_set_to_zero[df_flows_set_to_zero.Name == tech]['Activity'].unique()
    for act_name in act_name_list:
        act = [i for i in esm_db.db_as_list if i['name'] == act_name][0]
        direct_co2_emissions = sum([exc['amount'] for exc in act['exchanges'] if exc['name'] == 'Carbon dioxide, fossil'])
        direct_co2_emissions_dict[tech] += direct_co2_emissions

In [ ]:
# Characterization factors from IMPACT World+ (version 2.1)
cf_dict = {
    'Climate change, short term': 1.0,
    'Total human health': 7.197288788407747e-06,
    'Total ecosystem quality': 0.7025434445756051,
}

In [ ]:
esm_results_annual_prod['Direct CO2 emissions'] = esm_results_annual_prod['Annual_Prod'] * esm_results_annual_prod.Name.apply(lambda x: direct_co2_emissions_dict[x])

for imp_cat in ['Climate change, short term', 'Total human health', 'Total ecosystem quality']:
    # Adding direct CO2 emissions columns to the operation impacts dataframe
    esm_results_annual_prod[f'{imp_cat} (direct)'] = esm_results_annual_prod['Direct CO2 emissions'] * cf_dict[imp_cat]

    # The construction and resources impact are not modelled in ESMs without LCA
    esm_results_f_mult[f'{imp_cat} (direct)'] = 0
    esm_results_annual_res[f'{imp_cat} (direct)'] = 0

In [ ]:
esm_results_f_mult.to_csv('./results/soo_results_f_mult.csv', index=False)
esm_results_annual_prod.to_csv('./results/soo_results_annual_prod.csv', index=False)
esm_results_annual_res.to_csv('./results/soo_results_annual_res.csv', index=False)

## MOO

In [ ]:
# normalized limit = (physical limit [impact / cap] / max_AoP) * N_cap
N_run = 100
obj1 = 'TotalLCIA_m_CCS'
obj2 = 'TotalCost'

normalized_limits_list = []
obj1_min = main_variables_results[main_variables_results['Objective'] == obj1][obj1].values[0]
obj1_max = main_variables_results[main_variables_results['Objective'] == obj2][obj1].values[0]
normalized_limit_list = list(np.linspace(obj1_min, obj1_max, N_run))
# physical_limit_list = [i * max_ccs for i in normalized_limit_list]

In [ ]:
data = ['limit_lcia', obj1.split('TotalLCIA_')[-1], 2050, None, None] + normalized_limit_list
columns = ['param', 'index0', 'index1', 'index2', 'index3'] + [f'value{i+1}' for i in range(N_run)]

In [ ]:
seq_data = pd.DataFrame(data).T
seq_data.columns = columns

In [ ]:
es = run_esm(objective_function=obj2+'[2050]', returns='model')

In [ ]:
results_pareto = es.calc_sequence(seq_data)

## Visualize the results

In [ ]:
save_fig = False

In [ ]:
tech_to_show_list = [i for i in impact_scores.Name.unique() if i not in ['GRID']]

In [ ]:
esm_results_f_mult = pd.read_csv('./results/soo_results_f_mult.csv')
esm_results_annual_prod = pd.read_csv('./results/soo_results_annual_prod.csv')
esm_results_annual_res = pd.read_csv('./results/soo_results_annual_res.csv')

In [ ]:
run_first_it = ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']
run_second_it = [i for i in esm_results_f_mult.Run.unique() if i not in run_first_it]

### SOO

In [ ]:
mask_f_mult = (esm_results_f_mult.Run.isin(run_first_it)) & (esm_results_f_mult.Year == 2050)
mask_annual_prod = (esm_results_annual_prod.Run.isin(run_first_it)) & (esm_results_annual_prod.Year == 2050)
mask_annual_res = (esm_results_annual_res.Run.isin(run_first_it)) & (esm_results_annual_res.Year == 2050)

#### Installed capacities and annual productions

In [ ]:
df_cap = plot_energy_system_configuration(
    type='capacity',
    df_res=esm_results_f_mult[mask_f_mult],
    save_fig=save_fig,
)

In [ ]:
df_prod = plot_energy_system_configuration(
    type='production',
    df_res=esm_results_annual_prod[mask_annual_prod],
    save_fig=save_fig,
)

In [ ]:
df_syst_config = pd.merge(
    df_cap,
    df_prod,
    on=['Name', 'Run'],
    how='outer',
).rename(columns={'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'Run': 'Objective function', 'Name': 'Technology'})
df_syst_config['Objective function'] = df_syst_config['Objective function'].apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
df_syst_config = df_syst_config.sort_values(by=['Objective function', 'Technology'])[['Objective function', 'Technology', 'Installed capacity [GW]', 'Annual production [GWh]']]

#### Contribution per technology

In [ ]:
plot_technologies_contribution(
    cat='Total human health',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total ecosystem quality',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Climate change, short term',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total cost',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

Table of results

In [ ]:
list_esm_results_total = []

for obj in ['Total cost', 'Total human health', 'Total ecosystem quality', 'Climate change, short term']:
    esm_results_total = aggregate_phases_results(
        cat=obj,
        esm_results_f_mult=esm_results_f_mult[mask_f_mult],
        esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
        esm_results_annual_res=esm_results_annual_res[mask_annual_res],
        tech_to_show_list=tech_to_show_list,
    )
    esm_results_total['Metric'] = obj
    esm_results_total.rename(columns={obj: 'Value (total)', f'{obj} (direct)': 'Value (direct)', 'Name': 'Technology or resource'}, inplace=True)
    list_esm_results_total.append(esm_results_total)
esm_results_total = pd.concat(list_esm_results_total)
esm_results_total = esm_results_total[esm_results_total['Value (total)'] != 0]
esm_results_total.Run = esm_results_total.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_total.rename(columns={'Run': 'Objective function'}, inplace=True)
esm_results_total['Unit'] = esm_results_total.Metric.apply(lambda x: unit_ind_txt_dict[x]+'/(cap.yr)')
esm_results_total = esm_results_total.sort_values(['Metric', 'Objective function', 'Technology or resource'])[['Metric', 'Unit', 'Objective function', 'Technology or resource', 'Value (total)', 'Value (direct)']]

#### Contribution per impact category

In [ ]:
impact_scores = pd.read_csv(path_results+'impact_scores.csv')
impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

In [ ]:
hh_cat = [cat[2] for cat in impact_scores['Impact_category'].unique() if cat[1] == 'Human health' and cat[2] != 'Total human health']
eq_cat = [cat[2] for cat in impact_scores['Impact_category'].unique() if cat[1] == 'Ecosystem quality' and cat[2] != 'Total ecosystem quality']

In [ ]:
esm_results_f_mult_imp_cat_hh = esm_results_f_mult[mask_f_mult][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Construction').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_prod_imp_cat_hh = esm_results_annual_prod[mask_annual_prod][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Operation').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_res_imp_cat_hh = esm_results_annual_res[mask_annual_res][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Resource').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')

In [ ]:
esm_results_f_mult_imp_cat_eq = esm_results_f_mult[mask_f_mult][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Construction').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_prod_imp_cat_eq = esm_results_annual_prod[mask_annual_prod][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Operation').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_res_imp_cat_eq = esm_results_annual_res[mask_annual_res][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Resource').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')

In [ ]:
esm_results_imp_cat_hh = pd.merge(esm_results_f_mult_imp_cat_hh, esm_results_annual_prod_imp_cat_hh, on=['Run', 'Impact category'], how='outer')
esm_results_imp_cat_hh = pd.merge(esm_results_imp_cat_hh, esm_results_annual_res_imp_cat_hh, on=['Run', 'Impact category'], how='outer')

esm_results_imp_cat_eq = pd.merge(esm_results_f_mult_imp_cat_eq, esm_results_annual_prod_imp_cat_eq, on=['Run', 'Impact category'], how='outer')
esm_results_imp_cat_eq = pd.merge(esm_results_imp_cat_eq, esm_results_annual_res_imp_cat_eq, on=['Run', 'Impact category'], how='outer')

In [ ]:
esm_results_imp_cat_hh['Total'] = esm_results_imp_cat_hh['Construction'] + esm_results_imp_cat_hh['Operation'] + esm_results_imp_cat_hh['Resource']
esm_results_imp_cat_eq['Total'] = esm_results_imp_cat_eq['Construction'] + esm_results_imp_cat_eq['Operation'] + esm_results_imp_cat_eq['Resource']

In [ ]:
for col in ['Construction', 'Operation', 'Resource', 'Total']:
    esm_results_imp_cat_hh[col] = esm_results_imp_cat_hh[col] * 1e6 / N_cap
    esm_results_imp_cat_eq[col] = esm_results_imp_cat_eq[col] * 1e6 / N_cap

In [ ]:
esm_results_imp_cat_hh['Run'] = esm_results_imp_cat_hh['Run'].apply(lambda x: obj_code_dict[x])
esm_results_imp_cat_eq['Run'] = esm_results_imp_cat_eq['Run'].apply(lambda x: obj_code_dict[x])

In [ ]:
fig = px.bar(
    esm_results_imp_cat_hh,
    x='Run',
    y='Total',
    color='Impact category',
    barmode='stack',
    labels={
        'Impact category': 'Impact category',
        'Total': 'Human health damage [DALY/(cap.yr)]',
        'Run': 'Objective function',
    },
    width=650,
    height=525,
)

fig.for_each_trace(lambda t: t.update(marker_color=impact_category_hh_colors.get(t.name, '#000000')))
fig.update_layout(legend_traceorder='reversed')

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
)

if save_fig:
    fig.write_image('./figures/soo_impact_contrib_human_health.pdf')

fig.show()

In [ ]:
fig = px.bar(
    esm_results_imp_cat_eq,
    x='Run',
    y='Total',
    color='Impact category',
    barmode='stack',
    labels={
        'Impact category': 'Impact category',
        'Total': 'Ecosystem quality damage [PDF.m<sup>2</sup>.yr/(cap.yr)]',
        'Run': 'Objective function',
    },
    width=680,
    height=525,
)

fig.for_each_trace(lambda t: t.update(marker_color=impact_category_eq_colors.get(t.name, '#000000')))
fig.update_layout(legend_traceorder='reversed')

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
)

if save_fig:
    fig.write_image('./figures/soo_impact_contrib_ecosystem_quality.pdf')

fig.show()

Table of results

In [ ]:
esm_results_imp_cat_eq['Area of protection'] = 'Ecosystem quality'
esm_results_imp_cat_eq['Unit'] = 'PDF.m2.yr/(cap.yr)'
esm_results_imp_cat_hh['Area of protection'] = 'Human health'
esm_results_imp_cat_hh['Unit'] = 'DALY/(cap.yr)'
esm_results_imp_cat = pd.concat([
    esm_results_imp_cat_eq[['Area of protection', 'Unit', 'Run', 'Impact category', 'Total']],
    esm_results_imp_cat_hh[['Area of protection', 'Unit', 'Run', 'Impact category', 'Total']]
])
esm_results_imp_cat.Run = esm_results_imp_cat.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_imp_cat.rename(columns={'Total': 'Value', 'Run': 'Objective function'}, inplace=True)

#### Contribution per elementary flows

In [ ]:
contrib_analysis_ef = pd.read_csv(path_results+'2050/contribution_analysis_ef.csv')

In [ ]:
df_ef_contrib_eq = plot_ef_contributions(
    df_contrib_analysis_ef=contrib_analysis_ef,
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    main_variables_results=main_variables_results,
    aop='Total ecosystem quality',
    save_fig=save_fig,
)

In [ ]:
df_ef_contrib_hh =plot_ef_contributions(
    df_contrib_analysis_ef=contrib_analysis_ef,
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    main_variables_results=main_variables_results,
    aop='Total human health',
    save_fig=save_fig,
)

Table of results

In [ ]:
df_ef_contrib_eq['Area of protection'] = 'Ecosystem quality'
df_ef_contrib_eq['Unit'] = 'PDF.m2.yr/(cap.yr)'
df_ef_contrib_hh['Area of protection'] = 'Human health'
df_ef_contrib_hh['Unit'] = 'DALY/(cap.yr)'
df_ef_contrib = pd.concat([
    df_ef_contrib_eq[['Area of protection', 'Unit', 'Run', 'ef_name', 'scaled_impact']],
    df_ef_contrib_hh[['Area of protection', 'Unit', 'Run', 'ef_name', 'scaled_impact']]
])
df_ef_contrib.Run = df_ef_contrib.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
df_ef_contrib.rename(columns={'scaled_impact': 'Value', 'Run': 'Objective function', 'ef_name': 'Elementary flow'}, inplace=True)
df_ef_contrib.sort_values(['Area of protection', 'Objective function', 'Elementary flow'], inplace=True)

### SOO with ESM results

In [ ]:
esm_results_f_mult = pd.read_csv('./results/soo_results_f_mult.csv')
esm_results_annual_prod = pd.read_csv('./results/soo_results_annual_prod.csv')
esm_results_annual_res = pd.read_csv('./results/soo_results_annual_res.csv')

In [ ]:
df_cap_sec = plot_technologies_contribution_second_iteration(
    type='capacity',
    df_res=esm_results_f_mult,
    save_fig=save_fig,
)

In [ ]:
df_prod_sec = plot_technologies_contribution_second_iteration(
    type='production',
    df_res=esm_results_annual_prod,
    save_fig=save_fig,
)

In [ ]:
df_syst_config_sec = pd.merge(df_cap_sec[['Run', 'Background database', 'Name', 'F_Mult']], df_prod_sec[['Run', 'Background database', 'Name', 'Annual_Prod']])
df_syst_config_sec = df_syst_config_sec[df_syst_config_sec.F_Mult != 0]
df_syst_config_sec.rename(columns={'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'Run': 'Objective function', 'Name': 'Technology'}, inplace=True)
df_syst_config_sec['Objective function'] = df_syst_config_sec['Objective function'].apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)

In [ ]:
list_of_dfs = [esm_results_f_mult, esm_results_annual_prod, esm_results_annual_res]

for i, df in enumerate(list_of_dfs):
    filtered_df = df[df['Run'].isin(run_second_it)].copy()
    filtered_df['Background database'] = filtered_df['Run'].apply(harmonization_level)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_base', '', regex=False)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_esm_harmonized', '', regex=False)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_esm_not_harmonized', '', regex=False)
    list_of_dfs[i] = filtered_df

esm_results_f_mult, esm_results_annual_prod, esm_results_annual_res = list_of_dfs

In [ ]:
esm_results_total_tthh = aggregate_phases_results(
    cat='Total human health',
    esm_results_f_mult=esm_results_f_mult[(esm_results_f_mult.Year == 2050)],
    esm_results_annual_prod=esm_results_annual_prod[(esm_results_annual_prod.Year == 2050)],
    esm_results_annual_res=esm_results_annual_res[(esm_results_annual_res.Year == 2050)],
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Total human health',
    df_res=esm_results_total_tthh,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_tteq = aggregate_phases_results(
    cat='Total ecosystem quality',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Total ecosystem quality',
    df_res=esm_results_total_tteq,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_ccst = aggregate_phases_results(
    cat='Climate change, short term',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Climate change, short term',
    df_res=esm_results_total_ccst,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_tthh['Metric'] = 'Total human health'
esm_results_total_tteq['Metric'] = 'Total ecosystem quality'
esm_results_total_ccst['Metric'] = 'Climate change, short term'
esm_results_total_tthh['Unit'] = 'DALY/(cap.yr)'
esm_results_total_tteq['Unit'] = 'PDF.m2.yr/(cap.yr)'
esm_results_total_ccst['Unit'] = 't CO2-eq/(cap.yr)'

esm_results_total_tthh.rename(columns={'Total human health': 'Value (total)', 'Total human health (direct)': 'Value (direct)'}, inplace=True)
esm_results_total_tteq.rename(columns={'Total ecosystem quality': 'Value (total)', 'Total ecosystem quality (direct)': 'Value (direct)'}, inplace=True)
esm_results_total_ccst.rename(columns={'Climate change, short term': 'Value (total)', 'Climate change, short term (direct)': 'Value (direct)'}, inplace=True)

esm_results_total_sec = pd.concat([
    esm_results_total_tthh[['Metric', 'Unit', 'Run', 'Name', 'Value (total)', 'Value (direct)']],
    esm_results_total_tteq[['Metric', 'Unit', 'Run', 'Name', 'Value (total)', 'Value (direct)']],
    esm_results_total_ccst[['Metric', 'Unit', 'Run', 'Name', 'Value (total)', 'Value (direct)']],
])

esm_results_total_sec = esm_results_total_sec[esm_results_total_sec['Value (total)'] != 0]
esm_results_total_sec.Run = esm_results_total_sec.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_total_sec.rename(columns={'Run': 'Objective function', 'Name': 'Technology or resource'}, inplace=True)

esm_results_total_sec = esm_results_total_sec.sort_values(['Metric', 'Objective function', 'Technology or resource'])[['Metric', 'Unit', 'Objective function', 'Technology or resource', 'Value (total)', 'Value (direct)']]

### MOO

In [ ]:
plt.rcParams['font.family'] = 'arial'

#### Pareto front

In [ ]:
colors_var = 'TotalLCIA_TTHH'
add_soo_point = False

In [ ]:
x = [i * 1e3 * max_ccs / N_cap for i in normalized_limit_list]
y = list(results_pareto.variables[obj2].loc[2050][obj2])
colors = list(results_pareto.variables[colors_var].loc[2050][colors_var])

# adding data point from colors_var SOO
if add_soo_point:
    x = x + [main_variables_results[main_variables_results['Objective'] == colors_var][obj1].values[0]]
    y = y + [main_variables_results[main_variables_results['Objective'] == colors_var][obj2].values[0]]
    colors = colors + [main_variables_results[main_variables_results['Objective'] == colors_var][colors_var].values[0]]

# Create the scatter plot
scatter = plt.scatter(x, y, c=colors, cmap='jet')

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label(colors_var)

# Add labels and title
plt.xlabel(obj1)
plt.ylabel(obj2)

# Show the plot
plt.show()

In [ ]:
x = [i * 1e3 * max_ccs / N_cap for i in normalized_limit_list]
y_1 = list(results_pareto.variables['TotalCost'].loc[2050]['TotalCost'])
y_2 = list(results_pareto.variables['TotalLCIA_TTHH'].loc[2050]['TotalLCIA_TTHH'])
y_3 = list(results_pareto.variables['TotalLCIA_TTEQ'].loc[2050]['TotalLCIA_TTEQ'])
y_4 = list(results_pareto.variables['TotalLCIA_m_CCS'].loc[2050]['TotalLCIA_m_CCS'])

# Normalize the y values using the max value
y_1 = [i / max(y_1) for i in y_1]
y_2 = [i / max(y_2) for i in y_2]
y_3 = [i / max(y_3) for i in y_3]
y_4 = [i / max(y_4) for i in y_4]

plt.figure(figsize=(4.5, 3.5))

plt.plot(x, y_1, label='Total cost', c='k')
plt.plot(x, y_2, label='Total human health', c='k', ls='dashed')
plt.plot(x, y_3, label='Total ecosystem quality', c='k', ls='dotted')
# plt.plot(x, y_4, label='Climate change, short term', c='k', ls='dashdot')

plt.xlabel(f'Upper limit for {obj_name_dict[obj1].lower()} [t CO$_2$-eq/cap]')
plt.ylabel('Normalized indicator')

plt.legend()
plt.tight_layout()
if save_fig:
    plt.savefig('./figures/pareto_front_indicators.pdf')
plt.show()

In [ ]:
df_f_mult_pareto = results_pareto.variables['F_Mult'].reset_index()
x = [i * 1e3 * max_ccs / N_cap for i in normalized_limit_list]
y_1 = list(df_f_mult_pareto[(df_f_mult_pareto['index0'] == 'CCGT') & (df_f_mult_pareto['index1'] == 2050)]['F_Mult'])
y_2 = list(df_f_mult_pareto[(df_f_mult_pareto['index0'] == 'CCGT_CC') & (df_f_mult_pareto['index1'] == 2050)]['F_Mult'])
y_3 = list(df_f_mult_pareto[(df_f_mult_pareto['index0'] == 'PV') & (df_f_mult_pareto['index1'] == 2050)]['F_Mult'])

# from GW to kW / cap
y_1 = [i * 1e6 / N_cap for i in y_1]
y_2 = [i * 1e6 / N_cap for i in y_2]
y_3 = [i * 1e6 / N_cap for i in y_3]

plt.figure(figsize=(4.5, 3.5))

plt.plot(x, y_1, label=tech_name_dict['CCGT'], c=color_dict[tech_name_dict['CCGT']])
plt.plot(x, y_2, label=tech_name_dict['CCGT_CC'], c=color_dict[tech_name_dict['CCGT']], ls='-.')
plt.plot(x, y_3, label=tech_name_dict['PV'], c=color_dict[tech_name_dict['PV']])

plt.xlabel(f'Upper limit for {obj_name_dict[obj1].lower()} [t CO$_2$-eq/cap]')
plt.ylabel('Installed capacity [kW/cap]')

plt.legend()
plt.tight_layout()
if save_fig:
    plt.savefig('./figures/pareto_front_capacities.pdf')
plt.show()

### Save all dataframes to an Excel file

In [ ]:
with pd.ExcelWriter('results/SI_results_figures.xlsx', engine='xlsxwriter') as writer:
    workbook = writer.book

    # First iteration - energy system configurations
    writer.sheets['First it - System config'] = workbook.add_worksheet('First it - System config')
    df_syst_config.to_excel(writer, sheet_name='First it - System config', index=False, startrow=0, startcol=0)

    # First iteration - contributions of technologies
    writer.sheets['First it - Tech contributions'] = workbook.add_worksheet('First it - Tech contributions')
    esm_results_total.to_excel(writer, sheet_name='First it - Tech contributions', index=False)

    # First iteration - contributions of impact categories
    writer.sheets['First it - Impact categories'] = workbook.add_worksheet('First it - Impact categories')
    esm_results_imp_cat.to_excel(writer, sheet_name='First it - Impact categories', index=False)

    # First iteration - contributions of elementary flows
    writer.sheets['First it - Elementary flows'] = workbook.add_worksheet('First it - Elementary flows')
    df_ef_contrib.to_excel(writer, sheet_name='First it - Elementary flows', index=False)

    # Second iteration - energy system configurations
    writer.sheets['Second it - System config'] = workbook.add_worksheet('Second it - System config')
    df_syst_config_sec.to_excel(writer, sheet_name='Second it - System config', index=False)

    # Second iteration - contributions of technologies
    writer.sheets['Second it - Tech contributions'] = workbook.add_worksheet('Second it - Tech contributions')
    esm_results_total_sec.to_excel(writer, sheet_name='Second it - Tech contributions', index=False)